In [3]:
from pyspark.sql.session import SparkSession
import os
path = f"{os.getcwd()}/../"

from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, DateType, TimestampType
from pyspark.sql.functions import lit, col, from_json, regexp_replace, to_timestamp, current_timestamp, to_utc_timestamp

In [4]:
packages = [
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1",
    "org.apache.kafka:kafka-clients:3.2.1"
]

jars = [
    f"{path}/jars/commons-pool2-2.11.1.jar",
    f"{path}/jars/spark-sql-kafka-0-10_2.12-3.4.1.jar",
    f"{path}/jars/kafka-clients-3.5.1.jar",
    f"{path}/jars/spark-token-provider-kafka-0-10_2.12-3.4.1.jar",
    f"{path}/jars/mysql-connector-j-8.0.31.jar",
]

spark = (SparkSession
         .Builder()
         .appName(name="test_kafka")
         .master("spark://spark:7077")
         .config("spark.jars", ",".join(jars))
         .config("spark.jars.packages", ",".join(packages))
         #.config("spark.redis.host", "redis")
         #.config("spark.redis.port", "6379")
         .getOrCreate())

In [5]:
spark

In [6]:
spark.sparkContext.defaultParallelism

2

In [7]:
#spark.stop()

In [8]:
schema = StructType([
    StructField("id", IntegerType()),
    StructField("bme280", StructType([
        StructField("pressure", StringType()),
        StructField("temperature", StringType()),
        StructField("humidity", StringType()),
        StructField("read_datetime", StringType())
    ])),
    StructField("picow", StructType([
        StructField("local_ip", StringType()),
        StructField("temperature", DoubleType()),
        StructField("free_storage_kb", DoubleType()),
        StructField("mem_alloc_bytes", DoubleType()),
        StructField("mem_free_bytes", DoubleType()),
        StructField("cpu_freq_mhz", DoubleType())
    ]))
])

In [9]:
picow_df = (spark 
              .read
              .format("kafka") 
              .option("kafka.bootstrap.servers", "kafka:9092") 
              .option("subscribe", "iot_source")
              #.option("startingOffsets", "earliest")
              #.option("endingOffsets", "latest")
              .load())

df = (picow_df
      .where("offset > 2")
      .selectExpr("CAST(offset AS INTEGER)","CAST(timestamp AS TIMESTAMP)","CAST(key AS STRING)", "CAST(value AS STRING)", "CAST(topic AS STRING)"))

In [10]:
stg_df =  (df
            .withColumn('data', from_json(col('value'), schema))
            .selectExpr("offset as kafka_offset",
                        "timestamp as kafka_datetime",
                        "topic as kafka_topic",
                        "data.id as id",
                        "data.bme280.pressure as bme280_pressure",
                        "data.bme280.temperature as bme280_temperature",
                        "data.bme280.humidity as bme280_humidity",
                        "data.bme280.read_datetime as read_datetime",
                        "data.picow.local_ip as picow_local_ip",
                        "data.picow.temperature as picow_temperature",
                        "data.picow.mem_alloc_bytes as picow_mem_alloc_bytes",
                        "data.picow.mem_free_bytes as picow_mem_free_bytes",
                        "data.picow.cpu_freq_mhz as picow_cpu_freq_mhz",
                       )
            .drop("key","value"))

final_df = (stg_df
                .withColumn("kafka_datetime", to_utc_timestamp(col("kafka_datetime"), "UTC"))
                .withColumn("spark_process_datetime", current_timestamp())
                .withColumn("bme280_pressure",regexp_replace(col("bme280_pressure"),"hPa","").cast(DoubleType()))
                .withColumn("bme280_temperature",regexp_replace(col("bme280_temperature"),"C","").cast(DoubleType()))
                .withColumn("bme280_humidity",regexp_replace(col("bme280_humidity"),"%","").cast(DoubleType()))
                .withColumn("read_datetime", to_timestamp(col("read_datetime"), "yyyy-M-d HH:mm:s"))
                .withColumn("read_datetime", to_utc_timestamp(col("kafka_datetime"), "UTC"))
                .withColumn("read_date",col("read_datetime").cast("DATE"))
                .orderBy(col("read_datetime").desc())
                .where("id is not null")
           )

#final_df.printSchema()
#final_df.show(10,False)


In [11]:
final_df.count()

130899

In [23]:
final_df.where("read_date < '2023-10-14'").show(10,False)

+------------+-----------------------+-----------+--------+---------------+------------------+---------------+-----------------------+--------------+-----------------+---------------------+--------------------+------------------+--------------------------+----------+
|kafka_offset|kafka_datetime         |kafka_topic|id      |bme280_pressure|bme280_temperature|bme280_humidity|read_datetime          |picow_local_ip|picow_temperature|picow_mem_alloc_bytes|picow_mem_free_bytes|picow_cpu_freq_mhz|spark_process_datetime    |read_date |
+------------+-----------------------+-----------+--------+---------------+------------------+---------------+-----------------------+--------------+-----------------+---------------------+--------------------+------------------+--------------------------+----------+
|49138       |2023-10-13 23:59:57.659|iot_source |458679  |895.02         |24.48             |39.02          |2023-10-13 23:59:57.659|192.168.1.70  |27.0444          |150976.0             |41792.0

In [36]:
from pyspark.sql.functions import count, avg, round, month,year, dayofmonth, min, max, hour, desc

agg_by_hour_df = (
                 final_df
                 .withColumn("day", dayofmonth(col("read_date")))
                 .withColumn("month", month(col("read_date")))
                 .withColumn("year", year(col("read_date")))
                 
                 .groupBy("year", "month", "day", hour(col("read_datetime")).alias("hour"))
                 .agg(
                      count(col("id")).alias("count"),

                      # date columns
                      max(col("read_datetime")).alias("datetime"),
                      max(col("read_date")).alias("date"),
                      
                      # bme temperature metrics
                      round(min(col("bme280_temperature")),2).alias("min_bme_temp"),
                      round(avg(col("bme280_temperature")),2).alias("avg_bme_temp"),
                      round(max(col("bme280_temperature")),2).alias("max_bme_temp"),
                      

                      # picow temperature metrics
                      round(min(col("picow_temperature")),2).alias("min_picow_temp"),
                      round(avg(col("picow_temperature")),2).alias("avg_picow_temp"),
                      round(max(col("picow_temperature")),2).alias("max_picow_temp"),

                     # bme humidity metrics
                      round(min(col("bme280_humidity")),2).alias("min_bme_hum"),
                      round(avg(col("bme280_humidity")),2).alias("avg_bme_hum"),
                      round(max(col("bme280_humidity")),2).alias("max_bme_hum"),

                     # bme pressure metrics
                      round(min(col("bme280_pressure")),2).alias("min_bme_press"),
                      round(avg(col("bme280_pressure")),2).alias("avg_bme_press"),
                      round(max(col("bme280_pressure")),2).alias("max_bme_press")
                      
                     )
                 .sort(desc("year"), desc("month"), desc("day"), desc("hour"))
                 .selectExpr("datetime",
                             "date",
                             "year",
                             "month",
                             "day",
                             "hour",
                             "count",
                             "min_bme_temp",
                             "avg_bme_temp",
                             "max_bme_temp",
                             "min_picow_temp",
                             "avg_picow_temp",
                             "max_picow_temp",
                             "min_bme_hum",
                             "avg_bme_hum",
                             "max_bme_hum",
                             "min_bme_press",
                             "avg_bme_press",
                             "max_bme_press",
                            )
                )

In [80]:
mysql_tables = (spark.read 
                    .format("jdbc") 
                    .option("driver", "com.mysql.jdbc.Driver") 
                    .option("url", "jdbc:mysql://mariadb:3306/data?rewriteBatchedStatements=true") 
                    .option("user", "mysql") 
                    .option("password", "mysql") 
                    .option("dbtable", "information_schema.tables")
                    .load()
                    .filter("TABLE_SCHEMA = 'data'")
                    .select("TABLE_NAME")
                    .collect())

mysql_tables_list = [row["TABLE_NAME"] for row in mysql_tables]

if 'agg_by_hour_df' in mysql_tables_list:
    agg_by_hour_df_read = (spark.read 
                            .format("jdbc") 
                            .option("driver", "com.mysql.jdbc.Driver") 
                            .option("url", "jdbc:mysql://mariadb:3306/data?rewriteBatchedStatements=true") 
                            .option("user", "mysql") 
                            .option("password", "mysql") 
                            .option("dbtable", "agg_by_hour_df")
                            .load())
    
    agg_by_hour_df_combined = agg_by_hour_df_read.join(agg_by_hour_df,
                                                  agg_by_hour_df.datetime == agg_by_hour_df_read.datetime,
                                                  'inner')
    print(agg_by_hour_df_combined.count())
else:
    agg_by_hour_df_combined = agg_by_hour_df

0


In [93]:
agg_by_hour_df_read.printSchema()
agg_by_hour_df_read.show(10,False)
agg_by_hour_df.printSchema()
agg_by_hour_df.show(10,False)

root
 |-- datetime: timestamp (nullable = true)
 |-- date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- count: long (nullable = true)
 |-- min_bme_temp: double (nullable = true)
 |-- avg_bme_temp: double (nullable = true)
 |-- max_bme_temp: double (nullable = true)
 |-- min_picow_temp: double (nullable = true)
 |-- avg_picow_temp: double (nullable = true)
 |-- max_picow_temp: double (nullable = true)
 |-- min_bme_hum: double (nullable = true)
 |-- avg_bme_hum: double (nullable = true)
 |-- max_bme_hum: double (nullable = true)
 |-- min_bme_press: double (nullable = true)
 |-- avg_bme_press: double (nullable = true)
 |-- max_bme_press: double (nullable = true)

+-------------------+----------+----+-----+---+----+-----+------------+------------+------------+--------------+--------------+--------------+-----------+-----------+-----------+-------------+--------

In [38]:
(agg_by_hour_df.write 
        .format("jdbc") 
        .mode("overwrite") 
        .option("driver", "com.mysql.jdbc.Driver") 
        .option("url", "jdbc:mysql://mariadb:3306/data?rewriteBatchedStatements=true") 
        .option("user", "mysql") 
        .option("password", "mysql") 
        .option("dbtable", "agg_by_hour_df") 
        .save())

In [ ]:
agg_by_ip_df = (final_df
                .groupBy("picow_local_ip")
                .agg(count(col("id")).alias("messages"))
                .selectExpr("picow_local_ip as source_ip", "messages")
               )


In [ ]:
(agg_by_ip_df.write 
        .format("jdbc") 
        .mode("overwrite") 
        .option("driver", "com.mysql.jdbc.Driver") 
        .option("url", "jdbc:mysql://mariadb:3306/data?rewriteBatchedStatements=true") 
        .option("user", "mysql") 
        .option("password", "mysql") 
        .option("dbtable", "agg_by_ip_df") 
        .save())

In [29]:
pdf = agg_by_hour_df.toPandas()
pdf.to_csv(f"{path}data/agg_by_hour_df.csv", index=False)
pdf_ip = agg_by_ip_df.toPandas()
pdf_ip.to_csv(f"{path}data/agg_by_ip_df.csv", index=False)